In [1]:
# v2 scrapes from individual mod pages
from selenium import webdriver
import time
import json
import re
from tqdm import tqdm

def crawl():
    '''
    Collects list of mods on NUSMods from https://nusmods.com/modules before going to the individual module pages to 
    retrieve detailed information
    '''
    url = "https://nusmods.com/modules"
    modata = {}
    pg = 0
    with tqdm(total=1087) as pbar:
        while pg <=1087:
            driver = webdriver.Chrome(executable_path='/home/boshen/Projects/chromedriver')
            driver.get(url+"#page="+str(pg))
            time.sleep(3)
            modname=[]
            modId=[]
            isthereprereq=[]
            mod=driver.find_elements_by_css_selector('li.modules-item')
            for a in range(len(mod)):
                temp = mod[a].find_element_by_css_selector('h2.modules-title').text.split()
                try:
                    isthereprereq.append(mod[a].find_elements_by_css_selector('dl>dt')[-1].text)
                except:
                    isthereprereq.append("")
                modId.append(temp[0])
                modname.append(" ".join(temp[1:]))
            driver.quit()
            for i in range(len(modId)):
                if re.match(r'^[A-Z]{2,3}[0-9]{4}[A-Z]?$', modId[i])==None:
                    continue
                modata[modId[i]]={
                    "id": modId[i], 
                    "name": modname[i], 
                    "require": crawl2(modId[i]) if isthereprereq[i]=="Prerequisite" else []
                }
            pg+=2
            pbar.update(2)
    with open ("modata.json", 'a') as outfile:
        json.dump(modata, outfile)

    
def crawl2(mod):
    '''
    Retrieves information of a module from the https://nusmods.com/modules/{mod} page
    '''
    url = "https://nusmods.com/modules"
#     print("processing "+mod)
    driver = webdriver.Chrome(executable_path='/home/boshen/Projects/chromedriver')
    driver.get(url+"/"+mod)
    time.sleep(2)
    try:
        prereqtree = driver.find_element_by_css_selector("._352o-b9p > li._2W74AkOG")
                           .find_element_by_css_selector("div.hoverable + ul._1Xsx7hJQ")
                           .find_element_by_css_selector("li._2W74AkOG")
    except:
#         print(f"{mod} has no prerequisites?...")
        driver.quit()
        return []
    try: 
        topcmd = prereqtree.find_element_by_css_selector(" div._155WyYyd").text
        modGrps = prereqtree.find_elements_by_css_selector("ul._1Xsx7hJQ > li._2W74AkOG")
    except:
        topcmd=""
        modGrps = [prereqtree,]
    retlst = processLowLvl(topcmd, modGrps)
    driver.quit()
    return retlst

def processLowLvl(topcmd, HTMLelem):
    retlst=[]
    check_unique={""}
    if topcmd == "all of":
        for modGrp in HTMLelem:
            temp = {}
            mods = [mod.text for mod in modGrp.find_elements_by_css_selector("a._2koHWeSo")]
            realMods=[]
            for i in range(len(mods)):
                if mods[i] not in check_unique:
                    check_unique.add(mods[i])
                    realMods.append(mods[i])
            if len(realMods)==0:
                continue
            temp["name"]=realMods
            retlst.append(temp)
    elif topcmd == "one of" or topcmd=="":
        temp = {}
        for modGrp in HTMLelem:
            for mod in modGrp.find_elements_by_css_selector("a._2koHWeSo"):
                check_unique.add(mod.text)
        check_unique.remove("")
        temp["name"]=list(check_unique)
        retlst.append(temp)
    return retlst
            
            

In [2]:
crawl()
print("DONE!")

1088it [6:19:45, 29.19s/it]                          

DONE!


In [ ]:
# version 1: read only from nusmods.com/modules 
# from selenium import webdriver
# import time
# import json
# import re

        
# def parse(isthereprereq, _id, prereqstr):
# #     print(isthereprereq, _id)
#     if isthereprereq!="Prerequisite" or len(prereqstr)==0:
#         return []
#     if prereqstr[-1]=='.': 
#         prereqstr=prereqstr[:-1]
#     strlst = prereqstr.upper().replace('/', ' OR ').split(' ')
#     or_not_and = False
#     retlst=[]
#     for word in strlst:
#         if bool(re.search("^[A-Z]{2,3}[0-9]{4}[A-Z]?", word)):
#             if word==_id:
#                 continue
#             if or_not_and and len(retlst)!=0:
#                 retlst[-1]["name"].append(word)
#                 or_not_and=False
#             else:
#                 retlst.append({"name":[word,]})
#         elif word=="OR":
#             or_not_and=True
#     return retlst

# def crawl():
    
#     url = "https://nusmods.com/modules#page="
#     modata = {}
    
#     pg = 0
#     while pg <=1:#1083:
#         driver = webdriver.Chrome(executable_path='/home/boshen/Projects/chromedriver')
#         driver.get(url+str(pg))
#         time.sleep(3)
#         mod=driver.find_elements_by_css_selector('li.modules-item')
# #         print(mod[0].find_elements_by_css_selector('dl>dt')[-1].text)
#         for a in range(len(mod)):
#             modname = mod[a].find_element_by_css_selector('h2.modules-title').text.split()
            
#             try:
#                 isthereprereq = mod[a].find_elements_by_css_selector('dl>dt')[-1].text
#                 modprereqstr = mod[a].find_element_by_css_selector('dl dd:last-child').text
# #                 print(modprereqstr)
#             except:
#                 isthereprereq=''
#                 modprereqstr=''
#             modata[modname[0]]={
#                 "id": modname[0], "name": ' '.join(modname[1:]), "require": parse(isthereprereq,modname[0],modprereqstr)
#             }
#         print("processing page "+str(pg))
#         pg+=2
#         driver.quit()

# #     with open ("modata.json", 'a') as outfile:
# #         json.dump(modata, outfile)
#     print(modata)
# # if __name__ == "__main__":
# # #     options = webdriver.ChromeOptions()
# # #     options.add_argument('headless')
# # #     options.binary_location = '/home/boshen/Projects/chromedriver'
# #     crawl()
# #     print("done!")
